In [4]:
import CalculatedFieldSubroutines as cfs

#

import numpy as np

import pandas as pd

import random

#

import matplotlib.pyplot as plt

from pandasgui import show

#

import warnings

#

from sklearn.preprocessing import StandardScaler

from sklearn.neighbors import KNeighborsClassifier

import sklearn.metrics as metrics

In [5]:
warnings.filterwarnings( 'ignore' )

In [6]:
gmID_list = cfs.list_whitelisted_gmIDs_with_traffic_data()

topic_list = cfs.list_topics()

print( topic_list )

['/apollo/canbus/chassis', '/apollo/drive/event', '/apollo/sensor/gnss/best/pose', '/apollo/perception/traffic/light']


In [10]:
expansion_window = 1e9 #nanoseconds

#

red_preprocessed_dfs = []

green_preprocessed_dfs = []

blue_preprocessed_dfs = []

for gmID in gmID_list:

    preprocessed_df = cfs.retrieve_gmID_preprocessed_moving_data( gmID, window_seconds = 0 )

    #

    cfs.BinaryDisengagementExpanded( preprocessed_df, moving_colname = 'time', window = expansion_window )

    #

    cfs.DisengagementID( preprocessed_df, expanded = False )

    cfs.DisengagementID( preprocessed_df, expanded = True )

    #

    if ( cfs.give_route( gmID ) == 'Red' ):

        red_preprocessed_dfs.append( preprocessed_df )

    elif ( cfs.give_route( gmID ) == 'Green' ):

        green_preprocessed_dfs.append( preprocessed_df )

    elif ( cfs.give_route( gmID ) == 'Blue' ):

        blue_preprocessed_dfs.append( preprocessed_df )

In [33]:
red_preprocessed_dfs[ 0 ].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54648 entries, 0 to 54647
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Ind                          54648 non-null  int64  
 1   groupMetadataID              54648 non-null  object 
 2   time                         54648 non-null  int64  
 3   NormalizedTime               54648 non-null  float64
 4   BinaryDrivingMode            54648 non-null  int64  
 5   BinaryDisengagement          54648 non-null  int64  
 6   latitude                     54648 non-null  float64
 7   longitude                    54648 non-null  float64
 8   ProgressAlongRoute           54648 non-null  float64
 9   solStatus                    54648 non-null  object 
 10  solType                      54648 non-null  object 
 11  extendedSolutionStatus       54648 non-null  int64  
 12  numSatsInSolution            54648 non-null  int64  
 13  speedMps        

In [20]:
training_red_dfs, testing_red_dfs = cfs.random_list_split( red_preprocessed_dfs, split_percentage = 0.8 )

training_red_df = pd.concat( training_red_dfs )

testing_red_df = pd.concat( testing_red_dfs )

#

X_colnames = [ 'speedMps', 'brakePercentage', 'throttlePercentage', 'steeringPercentage', 'LatLonTotalStdDev', \
               'TernaryTurnSignal', 'BinaryContainLights' ]

Y_colname = 'BinaryDisengagementExpanded'

#

X_train = training_red_df[ X_colnames ]

Y_train = training_red_df[ Y_colname ]

#

X_test = testing_red_df[ X_colnames ]

Y_test = testing_red_df[ Y_colname ]

#

scaler = StandardScaler()

X_train = scaler.fit_transform( X_train )

X_test = scaler.transform( X_test )

#

knn = KNeighborsClassifier( n_neighbors = k )

knn.fit( X_train, Y_train )

Y_pred = knn.predict( X_test )

In [31]:
def ML_metrics( confusion_matrix_values, display = False ):

    TP, TN, FP, FN = confusion_matrix_values

    #

    precision = TP / ( TP + FP )

    recall = TP / ( TP + FN )

    tnr = TN / ( TN + FP )

    #

    accuracy = ( TP + TN ) / ( TP + TN + FP + FN )

    balanced_accuracy = ( recall + tnr ) / 2

    if ( display == True ):

        print( f'False_Negatives: { FN }, True_Positives: { TP }' )
    
        print( f'True_Negatives: { TN }, False_Positives: { FP }' )

        print( '' )

        print( f'Precision: { precision:.3f}' )

        print( f'Recall/True_Positive_Rate: { recall:.3f}' )

        print( f'True_Negative_Rate: { tnr:.3f}' )

        print( '' )

        print( f'Accuracy: { accuracy:.3f}' )

        print( f'Balanced_Accuracy: { balanced_accuracy:.3f}' )

    return TP, TN, FP, FN, precision, recall, tnr, accuracy, balanced_accuracy

In [73]:
def unique_disengagement_accuracy( Y_pred, Y_test, test_DisengagementExpandedID_col, display = False ):

    temp_df = pd.DataFrame()

    #

    temp_df[ 'Y_pred' ] = list( Y_pred ) 

    temp_df[ 'Y_test' ] = list( Y_test )

    #

    temp_df[ 'DisengagementExpandedID' ] = list( test_DisengagementExpandedID_col )

    #

    temp_df = temp_df[ temp_df[ 'DisengagementExpandedID' ] != 'NAD' ]

    #

    DisengagementID_col = []

    for ExpandedID in temp_df[ 'DisengagementExpandedID' ]:

        index = ExpandedID.index( 'ED' )

        ID = ExpandedID[ : index - 1 ]

        DisengagementID_col.append( ID )

    temp_df[ 'DisengagementID' ] = DisengagementID_col

    #

    unique_DisengagementIDs_wCounts = dict( temp_df[ 'DisengagementID' ].value_counts() )

    num_of_unique_DisengagementIDs = len( unique_DisengagementIDs_wCounts.keys() )

    #

    temp_df = temp_df[ ( temp_df[ 'Y_pred'] == 1 ) & ( temp_df[ 'Y_test' ] == 1 ) ]

    #

    unique_TP_DisengagementIDs_wCounts = dict( temp_df[ 'DisengagementID' ].value_counts() )

    num_of_unique_TP_DisengagementIDs = len( unique_TP_DisengagementIDs_wCounts.keys() )

    #

    unique_disengagement_accu = num_of_unique_TP_DisengagementIDs / num_of_unique_DisengagementIDs

    #

    individual_disengagement_accuracies = {}

    for ID in unique_DisengagementIDs_wCounts.keys():

        current_disengagement_count = unique_DisengagementIDs_wCounts[ ID ]

        try:

            current_TP_disengagement_count = unique_TP_DisengagementIDs_wCounts[ ID ]

        except:

            current_TP_disengagement_count = 0

        individual_disengagement_accuracies[ ID ] = current_TP_disengagement_count / current_disengagement_count

    individual_disengagement_accuracies = dict( sorted( individual_disengagement_accuracies.items(), \
                                                        key = lambda item : item[ 1 ], reverse = True ) )

    #

    if ( display == True ):

        print( f'Unique Disengagement Accuracy: { unique_disengagement_accu:.3f}' )

        print( f'# of Unique Testing Disengagements w/ a True Positive: { num_of_unique_TP_DisengagementIDs }' )

        print( f'# of Unique Testing Disengagements: { num_of_unique_DisengagementIDs }' )

    #

    return unique_disengagement_accu, num_of_unique_TP_DisengagementIDs, num_of_unique_DisengagementIDs, \
           individual_disengagement_accuracies, unique_TP_DisengagementIDs_wCounts, unique_DisengagementIDs_wCounts

In [74]:
metrics = ML_metrics( cfs.confusion_matrix_values( Y_pred, Y_test ), display = True )

False_Negatives: 3835, True_Positives: 405
True_Negatives: 745295, False_Positives: 3423

Precision: 0.106
Recall/True_Positive_Rate: 0.096
True_Negative_Rate: 0.995

Accuracy: 0.990
Balanced_Accuracy: 0.545


In [75]:
unique_disengagement_info = unique_disengagement_accuracy( Y_pred, Y_test, testing_red_df[ 'DisengagementExpandedID' ], display = True )

Unique Disengagement Accuracy: 0.415
# of Unique Testing Disengagements w/ a True Positive: 51
# of Unique Testing Disengagements: 123
